# GenAI Workshop
## Lesson 5: RAG Pitfalls

This lesson is intended to show you how different RAG configurations affect the output quality of a Retrieval Augmented Generation system.

During this lesson you will learn how to ...

- evaluate the quality of a RAG system using simple quality metrics
- manipulate the *chunk size* to improve the quality metrics
- manipulate *top k* to improve the quality metrics

### Set up the environment

In [ ]:
import os
import google.generativeai as genai

if os.getenv("COLAB_RELEASE_TAG"):
   COLAB = True
   print("Running on COLAB environment.")
else:
   COLAB = False
   print("WARNING: Running on LOCAL environment.")


In [ ]:
# Clone the data repository into colab
!git clone https://github.com/openknowledge/workshop-genai-data.git
PROCESSED_DATA_PATH = "/content/workshop-genai-data/processed/gutenberg/"
EVALUATION_DATA_PATH = "/content/workshop-genai-data/evaluation/"

In [ ]:
# import colab specific lib to read user data (aka colab managed secrets)
from google.colab import userdata

In [ ]:
# Initialize Google GenAI Client API with GOOGLE_API_KEY to be able to call the model.
# Note: GEMINI_API_KEY must be set as COLAB userdata before!
GOOGLE_API_KEY=userdata.get('GEMINI_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
# Install additional libraries
%%capture
!pip install -qU langchain-text-splitters
!pip install chromadb

In [ ]:
# Import additional libraries
from langchain_text_splitters import RecursiveCharacterTextSplitter
from chromadb import EphemeralClient
import requests
import re
import uuid
import json
import typing_extensions as typing
from google.generativeai.types import HarmCategory, HarmBlockThreshold
import pandas as pd
from pathlib import Path
import time
import numpy as np


In [ ]:
# Configure pandas display options
pd.set_option("max_colwidth", None)

In [ ]:
# Set default values for model, model parameters and prompt
DEFAULT_MODEL = "gemini-1.5-flash"
DEFAULT_CONFIG_TEMPERATURE = 0.0
DEFAULT_CONFIG_TOP_K = 1
DEFAULT_CONFIG_MAX_OUTPUT_TOKENS = 200
DEFAULT_SYSTEM_PROMPT = "Your are a friendly assistant"
DEFAULT_USER_PROMPT = " "

# Set defaults for retrieval
DEFAULT_TOP_K = 3
DEFAULT_CHUNK_OVERLAP = 100
DEFAULT_CHUNK_SIZE = 2000

In [ ]:
# This will be the chromadb collection we use as a knowledge base. We do not need the in-memory EphemeralClient.
chromadb_client = EphemeralClient()
chromadb_collection = chromadb_client.create_collection(name="default")


In [ ]:
def call_genai_model_for_completion(
    model_name: str = DEFAULT_MODEL,
    config_temperature:float = DEFAULT_CONFIG_TEMPERATURE,
    config_top_k: int = DEFAULT_CONFIG_TOP_K,
    config_max_output_tokens: int = DEFAULT_CONFIG_MAX_OUTPUT_TOKENS,
    system_prompt : str = DEFAULT_SYSTEM_PROMPT,
    user_prompt : str = DEFAULT_USER_PROMPT,
    verbose: bool = False
    ):
    """ Calls a gemini model with a given set of parameters and returns the completions

    Parameters
    ----------
    model_name : str, optional [default: DEFAULT_MODEL]
        The name of the model to use for the completion
    temperature : float, optional [default: DEFAULT_CONFIG_TEMPERATURE]
        The temperature of the model
    top_k : int, optional [default: DEFAULT_CONFIG_TOP_K]
        The number of most recent matches to return
    max_output_tokens : int, optional [default: DEFAULT_CONFIG_MAX_OUTPUT_TOKENS]
        The maximum number of output tokens to return
    system_prompt : str, optional [default: DEFAULT_SYSTEM_PROMPT]
        The system prompt to use for the completion
    user_prompt : str, optional [default: DEFAULT_USER_PROMPT]
        The user prompt to use for the completion
    file_list : [str], optional [default: empty list]
    verbose : bool, optional [default: False]
        Whether to print details of the completion process or not. Defaults to False
    Returns
    -------
    completions :
        a GenerateContentResponse instance representing the genAI model answer(s)
    """

    if verbose:
        # print out summary of input values / parameters
        print(f'Generating answer for following config:')
        print(f'  - SYSTEM PROMPT used:\n {system_prompt}')
        print(f'  - USER PROMPT used:\n {user_prompt}')
        print(f'  - MODEL used:\n {model_name} (temperature = {config_temperature}, top_k = {config_top_k}, max_output_tokens = {config_max_output_tokens})')

    # create generation config
    model_config = genai.GenerationConfig(
        max_output_tokens=config_max_output_tokens,
        temperature=config_temperature,
        top_k=config_top_k
    )

    # create genai model with generation config
    genai_model = genai.GenerativeModel(
        model_name= model_name,
        generation_config= model_config
    )

    # Attention: We manipulated the safety settings
    response = genai_model.generate_content(
        contents=[system_prompt, user_prompt], safety_settings={
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
    })
    return response

In [ ]:
# RAG building blocks

# Get content of books. The content will already be cleansed.
def load_file_content(file_name: str) -> str:
  """ Loads content of a file in the local file systemby a given file name and return its content

    Parameters
    ----------
    file_name : str
        The name of the file to be loaded
    Returns
    -------
    file_content : str
        the content of the file as a string
  """
  with open(f"{PROCESSED_DATA_PATH}{file_name}", "r") as f:
    return f.read()

# Building Block "Chunking": Split the content into smaller chunks
def do_chunk(text: str, chunk_size: int = DEFAULT_CHUNK_SIZE, chunk_overlap: int = DEFAULT_CHUNK_OVERLAP) -> list[str]:
  """ Chunks a given text by a given chunk size and chunk overlap and returns a list of chunks

  Parameters
  ----------
  text : str
      The text to be chunked
  chunk_size : int, optional [default: DEFAULT_CHUNK_SIZE]
        The desired chunk size
  chunk_overlap : int, optional [default: DEFAULT_CHUNK_OVERLAP]
        The desired chunk overlap
  Returns
  -------
  chunks: [str]
      The created chunks
  """
  text_splitter = RecursiveCharacterTextSplitter(
      chunk_size=chunk_size,
      chunk_overlap=chunk_overlap,
      length_function=len,
  )
  return text_splitter.split_text(text=text)

# Building Block "Embedding": Create multi dimensional embeddings for a given chunk.
def do_embed(chunk: str) -> list[float]:
  """ Embeds a given chunk and returns the embedding

  Parameters
  ----------
  chunk : str
      The chunk to be embedded
  Returns
  -------
  embedding: [float]
      The created embedding
  """
  return genai.embed_content(model=EMBEDDING_MODEL, content=chunk).get("embedding")

def do_batch_embed(chunks: list[str]) -> list[list[float]]:
  """ Embeds a list of chunks and returns the embeddings

  Parameters
  ----------
  chunks : [str]
      The chunks to be embedded
  Returns
  -------
  embeddings: [list[float]]
      The created embeddings
  """
  return genai.embed_content(model=EMBEDDING_MODEL, content=chunks).get("embedding")

# Building Block "Knowledgebase": Store embeddings and the corresponding content in a vectorstore
def persist_embeddings(chunks: list[str], embeddings: list[float])-> None:
  """ Persists the embeddings and the chunks in the knowledgebase

  Parameters
  ----------
  chunks : [str]
      The chunks to be persisted
  embeddings: [list[float]]
      The corresponding embeddings to be persisted
  """
  chromadb_collection = chromadb_client.get_or_create_collection(name="default")
  # Persist the embeddings and the chunks in the knowledgebase
  ids = [str(uuid.uuid4()) for _ in chunks]
  chromadb_collection.add(ids=ids, documents=chunks, embeddings=embeddings)

# Building Block "Augmentation": Create an updated prompt by merging the original user input with the provided context
# Attention: We manipulated the augmented prompt in order to see the guardrails in action
def augment(user_input: str, context: list[str]) -> str:
  """ Augments a given user input by merging it with the provided context and returns the augmented prompt

  Parameters
  ----------
  user_input : str
      The user input to be augmented
  context : [str]
      The context to be merged with the user input
  Returns
  -------
  augmented_prompt: str
      The created augmented prompt
  """
  prepared_context = "\n".join(context)
  augmented_prompt = f"""
    Answer the question as detailed as possible from the provided context. If the answer is not in
    provided context just say, 'answer is not available in the context', don't provide the wrong answer.
    Respond short and concisely.
    Context:\n{prepared_context}?\n
    Question: \n{user_input}\n

    Answer:
  """
  return augmented_prompt

# Building Block "Top-k Fetching": Get the k semantically closest chunks to the user input from the knowledgebase
def do_top_k_fetching(user_input_embedding: list[float], top_k: int) -> tuple[list[str],list[float]]:
  """ Fetches the k semantically closest chunks to the user input from the knowledgebase

  Parameters
  ----------
  user_input_embedding : [float]
      The embedding of the user input
  top_k : int
      The number of semantically closest chunks to be fetched

  Returns
  -------
  context: [str]
      The fetched chunks
  distances: [float]
      The corresponding distances to the user_input_embedding
  """
  # Since we will do the fetching always only for one user_input,
  # instead of querying for multiple embeddings simultanously as allowed by the choma API,
  # we add the embeddings below to a list and return only the first document (chunk)
  chromadb_collection = chromadb_client.get_or_create_collection(name="default")
  results = chromadb_collection.query(query_embeddings=[user_input_embedding], n_results=top_k)
  return (results["documents"][0], results["distances"][0]) # Return the distances to get better insights

# Building Block "Generation": Use the generation model to create a response
# We return the total token count for this exercise
def generate_response(prompt: str) -> tuple[str, int]:
  """ Generates a response for a given prompt

  Parameters
  ----------
  prompt : str
      The prompt to be used for the generation
  Returns
  -------
  response: str
      The generated response
  total_token_count: int
      The total token count for the generation
  """
  completion_result = call_genai_model_for_completion(
      model_name=GENERATION_MODEL,
      user_prompt=prompt,
  )
  total_token_count = completion_result.usage_metadata.total_token_count
  return (completion_result.text, total_token_count)

In [ ]:
def do_ingestion(file_names: list[str], chunk_size: int = DEFAULT_CHUNK_SIZE, clear_knowledgebase: bool = False, verbose: bool = False) -> None:
  """ Ingests a list of files by a given file name and chunk

  Parameters
  ----------
  file_names : [str]
      The names of the files to be ingested
  chunk_size : int, optional [default: DEFAULT_CHUNK_SIZE]
      The desired chunk size
  clear_knowledgebase : bool, optional [default: False]
      Whether to clear the knowledgebase before ingesting the new files
  verbose : bool, optional [default: False]
      Whether to print details of the ingestion process or not. Defaults to False
  """
  if clear_knowledgebase:
    chromadb_client.delete_collection(name="default")
  # Ingest file by file
  for file_name in file_names:
    # Load prepared book content
    file_content = load_file_content(file_name)

    # Chunk the content into smaller chunks
    chunks = do_chunk(file_content, chunk_size=chunk_size)
    if verbose:
      print(f'Loaded {len(chunks)} chunks from {file_name}')

    # Embed the chunks
    embeddings = do_batch_embed(chunks)

    # Persist the embeddings and the chunks in the knowledgebase
    persist_embeddings(chunks, embeddings)

  if verbose:
    chunks_count = chromadb_client.get_collection(name="default").count()
    print(f'Added {chunks_count} chunks to the knowledgebase')

In [ ]:
def do_rag(user_input: str, top_k: int = DEFAULT_TOP_K, verbose: bool = False) -> tuple[str, list[str], list[float], int]:
  """ Runs the RAG pipeline with a given user input

  Parameters
  ----------
  user_input : str
      The user input to be used for the RAG pipeline
  top_k : int, optional [default: DEFAULT_TOP_K]
      The number of semantically closest chunks to be fetched
  verbose : bool, optional [default: False]
      Whether to print details of the RAG process or not. Defaults to False
  Returns
  -------
  response: str
      The generated response
  context: [str]
      The fetched chunks
  distances: [float]
      The corresponding distances to the user_input_embedding
  """
  # Embed the user input
  user_input_embedding = do_embed(chunk=user_input)

  # "R" like "Retrieval": Get the k semantically closest chunks to the user input from the knowledgebase
  retrieval_start_time = time.time()
  (context, distances) = do_top_k_fetching(user_input_embedding=user_input_embedding, top_k=top_k)
  if verbose:
    retrieval_end_time = time.time()
    retrieval_time = round(retrieval_end_time - retrieval_start_time, 2)

  # "A" like "Augmented": Create the augmented prompt
  augmented_prompt = augment(user_input=user_input, context=context)

  # "G" like "Generation": Generate a response
  generation_start_time = time.time()
  (response, total_token_count) = generate_response(prompt=augmented_prompt)
  if verbose:
    generation_end_time = time.time()
    generation_time = round(generation_end_time - generation_start_time, 2)
    print(f'Retrieval took {retrieval_time}s. Generation took {generation_time}s.')

  return (response, context, distances, total_token_count)


In [ ]:
# Define a custom exception
class FactCheckingValidationError(Exception):
  """ Exception raised for errors in the fact checking validation. """
  pass

# Define a response format
class FactCheckingValidationAnswer(typing.TypedDict):
  """ Response format for the fact checking validation. """
  is_grounded: bool


def guard_fact_checking(bot_response: str, context: list[str]) -> str:
  """ Uses fact checking validation for a given bot response and context

  Parameters
  ----------
  bot_response : str
      The bot response to be guarded
  context : [str]
      The context to be used for the fact checking validation
  Returns
  -------
  bot_response: str
      The guarded bot response
  """
  # Prepare the context to be used in the guard prompt
  context = "\n".join(context)

  # Define the prompt for the guardrail
  guard_prompt = f"""
    You are given a task to identify if the answer is grounded and entailed to the context.
    You will only use the contents of the context and not rely on external knowledge.
    'context': {context} 'answer': {bot_response}
    """

  # Call the guardrail model with the desired output format
  model = genai.GenerativeModel(GUARDING_MODEL)
  result = model.generate_content(
      guard_prompt,
      generation_config=genai.GenerationConfig(
          response_mime_type="application/json", response_schema=FactCheckingValidationAnswer
      ),
  )

  # Evaluate the validation
  fact_checking_validation = json.loads(result.text)
  if not fact_checking_validation["is_grounded"]:
    error_msg = f"The bot answer '{bot_response}' is not grounded in the context '{context}'"
    raise FactCheckingValidationError(error_msg)
  return bot_response

In [ ]:
def do_fact_checked_rag(user_input: str, top_k: int = DEFAULT_TOP_K, verbose: bool = False) -> tuple[str, list[str], list[float], int]:
  """ Runs the RAG pipeline with a given user input and fact checking validation

  Parameters
  ----------
  user_input : str
      The user input to be used for the RAG pipeline
  top_k : int, optional [default: DEFAULT_TOP_K]
      The number of semantically closest chunks to be fetched
  verbose : bool, optional [default: False]
      Whether to print details of the RAG process or not. Defaults to False
  Returns
  -------
  response: str
      The generated response
  context: [str]
      The fetched chunks
  distances: [float]
      The corresponding distances to the user_input
  total_token_count: int
      The total token count for the generation
  """
  (answer, context, distances, total_token_count) = do_rag(user_input=user_input, top_k=top_k, verbose=verbose)
  try:
    guarded_output = guard_fact_checking(bot_response=answer, context=context)
  except FactCheckingValidationError:
    # Return a predefined response if the fact checking validation fails
    guarded_output = "Possible hallucination detected."
  return (guarded_output, context, distances, total_token_count)

In [ ]:
def print_insights(dataframe: pd.DataFrame):
  """ Prints the insights for a given evaluation dataframe

  Parameters
  ----------
  dataframe : pd.DataFrame
      The dataframe to be analyzed
  """
  median_response_time = round(dataframe['response_time'].median(), 2)
  median_min_distance = round(dataframe['min_context_distance'].median(), 2)
  median_median_distance = round(dataframe['median_context_distance'].median(), 2)
  median_total_token_count = round(dataframe['total_token_count'].median(), 2)
  n_hallucinations = dataframe["rag_response"].value_counts().get("Possible hallucination detected.", 0)

  print(f'Number of detected hallucinations: {n_hallucinations}')
  print(f'Median response time: {median_response_time} seconds')
  print(f'Median of minimum distance: {median_min_distance}')
  print(f'Median of median distance: {median_median_distance}')
  print(f'Median total token count: {median_total_token_count}')

In [ ]:
# The function gives us the desired outputs to gather some insights
def generate_rag_answers(dataframe: pd.DataFrame, top_k: int = DEFAULT_TOP_K, verbose: bool = False):
  """ Generates the RAG answers for a given dataframe

  Parameters
  ----------
  dataframe : pd.DataFrame
      The dataframe, which includes the questions to be answered
  top_k : int, optional [default: DEFAULT_TOP_K]
      The number of semantically closest chunks to be fetched
  verbose : bool, optional [default: False]
      Whether to print details of the RAG process or not. Defaults to False
  Returns
  -------
  dataframe: pd.DataFrame
      The dataframe with the RAG answers
  """
  def generate_rag_response_with_insights(question: str) -> pd.Series:
        # Define start time for calculating the response time
        start_time = time.time()

        # Generate the RAG response
        output = do_fact_checked_rag(question, top_k=top_k, verbose=verbose)
        response = output[0]  # Extract the response from the output
        distances = output[2]  # Extract the distances from the output
        total_token_count = output[3]  # Extract the total token count from the output
        min_distance = round(min(distances), 2)  # Find the minimum distance
        median_distance = np.nanmedian(distances)  # Calculate the mean distance

        # Define end time for calculating the response time
        end_time = time.time()

        # Calculate response time
        response_time = round(end_time - start_time, 2)

        # Return response and insights
        return pd.Series([response, response_time, min_distance, median_distance, total_token_count])

  # Apply the insights function to each row and store results in new columns
  dataframe[['rag_response', 'response_time', 'min_context_distance', 'median_context_distance', 'total_token_count']] = dataframe['question'].apply(generate_rag_response_with_insights)
  return dataframe


### Configure the genAI models

In [ ]:
GENERATION_MODEL = "gemini-1.5-flash"
EMBEDDING_MODEL = "models/text-embedding-004"
GUARDING_MODEL = "gemini-1.5-flash-8b"

### Prepare the knowledgebase

In [ ]:
KNOWLEDGEBASE_CONTENT = set(['study_in_scarlett.txt'])

do_ingestion(file_names=KNOWLEDGEBASE_CONTENT)

### Manually evaluate responses

In [ ]:
# Read csv from local files
evaluation_dataframe = pd.read_csv(EVALUATION_DATA_PATH + 'simple_evaluation_dataset.csv')
evaluation_dataframe.head()

In [ ]:
# Generate responses and insights. We use a wrapper function, which collects the insights and stores them in a dataframe
evaluation_dataframe = generate_rag_answers(evaluation_dataframe)
evaluation_dataframe

In [ ]:
print_insights(evaluation_dataframe)

### Ingest remaining stories

In [ ]:
# Ingest the second book, which contains the other stories
KNOWLEDGEBASE_CONTENT.add('adventures_of_sherlock_holmes.txt')
do_ingestion(file_names=KNOWLEDGEBASE_CONTENT, clear_knowledgebase=False)

In [ ]:
# Generate responses and insights
evaluation_dataframe = generate_rag_answers(evaluation_dataframe)
evaluation_dataframe

In [ ]:
print_insights(evaluation_dataframe)

### Manually adjust *chunk size*

In [ ]:
# Define new chunk size. Be careful: The smaller the chunk_size, the more time is needed for the ingestion. I.e. adjusted_chunk_size = 500 takes 1-2 minutes
adjusted_chunk_size = 1000

In [ ]:
# We need to clear the knowledgebase for this
do_ingestion(file_names=KNOWLEDGEBASE_CONTENT, chunk_size=adjusted_chunk_size, clear_knowledgebase=True, verbose=True)

In [ ]:
# Generate responses and insights
evaluation_dataframe = generate_rag_answers(evaluation_dataframe, verbose=True)
evaluation_dataframe

In [ ]:
print_insights(evaluation_dataframe)

### Manually adjust *top_k*
We do not need to ingest again, since our adjustment just affects the retrieval pipeline. Keep in mind, that we use the chunking from before, because we did not update the knowledgebase.


In [ ]:
adjusted_top_k = 10

In [ ]:
# Generate responses and insights
evaluation_dataframe = generate_rag_answers(evaluation_dataframe, top_k=adjusted_top_k, verbose=True)
evaluation_dataframe

In [ ]:
print_insights(evaluation_dataframe)

### Exercise 01: Find the best parameter values for *chunk_size* and *top_k*
- **Main goal**: Generate a minimum amount of hallucinations
- **Second goal**: Keep the total count of tokens as small as possible to reduce costs

In [ ]:
# We use a smaller set of evaluation question to reduce response time
evaluation_dataframe = pd.read_csv(EVALUATION_DATA_PATH + 'simple_evaluation_dataset_shortened.csv')
evaluation_dataframe.head()

In [ ]:
# The knowledgebase consists of shortened versions of the books to reduce ingestion time. Do not change this.
KNOWLEDGEBASE_CONTENT = ['study_in_scarlett_shortened.txt', 'adventures_of_sherlock_holmes_shortened.txt']

In [ ]:
# TODO: Adjust the following parameters
chunk_size = None
top_k = None

In [ ]:
# This needs to be executed if you changed chunk_size
do_ingestion(file_names=KNOWLEDGEBASE_CONTENT, chunk_size=chunk_size)

In [ ]:
# This needs to be executed always
evaluation_dataframe = generate_rag_answers(evaluation_dataframe, top_k=top_k)
evaluation_dataframe

In [ ]:
# TODO: Analyze the insights and iteratively find the best configuration for chunk_size and top_k
print_insights(evaluation_dataframe)